In [1]:
import os
import pandas as pd

In [2]:
EXP_DIR = './exp_baseline_submission'
EXPERIMENT_NAME = 'hb3cfb583ad03be38'
SUBM_DATA_DIR = './data/subm_cfp'

submission_prediction_fnames = [fn for fn in os.listdir(EXP_DIR) if fn.startswith('submission_predictions_')]
submission_prediction_fnames

['submission_predictions_hb3cfb583ad03be38_s300_f2.csv',
 'submission_predictions_hb3cfb583ad03be38_s200_f1.csv',
 'submission_predictions_hb3cfb583ad03be38_s100_f0.csv',
 'submission_predictions_hb3cfb583ad03be38_s500_f4.csv',
 'submission_predictions_hb3cfb583ad03be38_s400_f3.csv']

In [3]:
submission_files = [fn for fn in os.listdir(SUBM_DATA_DIR)]
submission_files

['TEST08525.jpg',
 'TEST03423.jpg',
 'TEST06866.jpg',
 'TEST08419.jpg',
 'TEST08187.jpg',
 'TEST07609.jpg',
 'TEST09218.jpg',
 'TEST06561.jpg',
 'TEST01442.jpg',
 'TEST01277.jpg',
 'TEST00539.jpg',
 'TEST07485.jpg',
 'TEST04131.jpg',
 'TEST08502.jpg',
 'TEST02167.jpg',
 'TEST07887.jpg',
 'TEST04623.jpg',
 'TEST02831.jpg',
 'TEST04988.jpg',
 'TEST05727.jpg',
 'TEST00563.jpg',
 'TEST04601.jpg',
 'TEST00201.jpg',
 'TEST04372.jpg',
 'TEST02157.jpg',
 'TEST08476.jpg',
 'TEST03379.jpg',
 'TEST09149.jpg',
 'TEST01624.jpg',
 'TEST00497.jpg',
 'TEST06499.jpg',
 'TEST02630.jpg',
 'TEST09126.jpg',
 'TEST09511.jpg',
 'TEST09050.jpg',
 'TEST01674.jpg',
 'TEST09971.jpg',
 'TEST07541.jpg',
 'TEST07851.jpg',
 'TEST05220.jpg',
 'TEST06666.jpg',
 'TEST00155.jpg',
 'TEST07289.jpg',
 'TEST09838.jpg',
 'TEST01016.jpg',
 'TEST09592.jpg',
 'TEST07191.jpg',
 'TEST06076.jpg',
 'TEST04206.jpg',
 'TEST06765.jpg',
 'TEST00420.jpg',
 'TEST02824.jpg',
 'TEST04002.jpg',
 'TEST08485.jpg',
 'TEST09864.jpg',
 'TEST0319

In [4]:
df_sp_list = [pd.read_csv(os.path.join(EXP_DIR, fn), index_col=0) for fn in submission_prediction_fnames]
df_sp_list[0]


,filename,predictions
0,TEST00000.png,0.999248
1,TEST00001.png,0.000095
2,TEST00002.png,0.000206
3,TEST00003.png,0.999012
4,TEST00004.png,0.000294
...,...,...
9,TEST09995.png,0.000327
10,TEST09996.png,0.000362
11,TEST09997.png,0.000778
12,TEST09998.png,0.740718


In [5]:
import numpy as np
submission_df = pd.DataFrame(data={'aimi_id':np.array([fn[:-4] for fn in submission_files])})

In [6]:
submission_df

,aimi_id
0,TEST08525
1,TEST03423
2,TEST06866
3,TEST08419
4,TEST08187
...,...
9995,TEST04154
9996,TEST07878
9997,TEST04189
9998,TEST00909


In [7]:
for i, df in enumerate(df_sp_list):
    df['aimi_id'] = df['filename'].apply(lambda x: x[:-4])
    df.rename(columns={'predictions':f'rg_likelihood_{i}'}, inplace=True)
    df.drop(columns='filename', inplace=True)
    df.set_index('aimi_id', inplace=True)

submission_df.set_index('aimi_id', inplace=True)

In [8]:
df_sp_list[0]

,rg_likelihood_0
aimi_id,
TEST00000,0.999248
TEST00001,0.000095
TEST00002,0.000206
TEST00003,0.999012
TEST00004,0.000294
...,...
TEST09995,0.000327
TEST09996,0.000362
TEST09997,0.000778


In [9]:
submission_df_merged = submission_df.copy()
for df in df_sp_list:
    submission_df_merged = pd.merge(submission_df_merged, df, how='left', on='aimi_id')

In [10]:
submission_df_merged['rg_likelihood'] = submission_df_merged.iloc[:, 1:].mean(axis=1)
submission_df_merged

,rg_likelihood_0,rg_likelihood_1,rg_likelihood_2,rg_likelihood_3,rg_likelihood_4,rg_likelihood
aimi_id,,,,,,
TEST08525,0.000805,0.000390,0.002201,0.000273,0.001630,0.001124
TEST03423,0.000343,0.000134,0.020514,0.000143,0.000115,0.005227
TEST06866,0.000103,0.000325,0.000275,0.000127,0.000165,0.000223
TEST08419,0.000129,0.001908,0.011932,0.000180,0.002235,0.004064
TEST08187,0.000277,0.000278,0.000308,0.000082,0.000127,0.000199
...,...,...,...,...,...,...
TEST04154,0.537611,0.161005,0.363858,0.001932,0.077031,0.150956
TEST07878,0.000566,0.001096,0.001268,0.000221,0.000434,0.000755
TEST04189,0.000336,0.000444,0.001031,0.000131,0.000229,0.000459


In [22]:
to_submit = submission_df_merged.sort_index(ascending=True)['rg_likelihood']
to_submit[to_submit.isna()]

aimi_id
TEST01971   NaN
TEST08978   NaN
Name: rg_likelihood, dtype: float64

In [24]:
to_submit['TEST01971'] = 0.004
to_submit['TEST08978'] = 0.3

In [25]:
to_submit.to_csv('submission.csv', sep=';')

In [1]:
(0.93049 + 0.93408 + 0.93271)/3

0.9324266666666667

In [2]:
(0.89353 + 0.90920 + 0.90565) / 3

0.9027933333333333

In [3]:
(0.93302 + 0.92680 + 0.93179) / 3

0.9305366666666667

In [5]:
(0.90355 + 0.90122 + 0.87590) / 3

0.8935566666666667

In [7]:
# baseline (tuned yolo) 1.2 crop
round((0.92960+0.93282+0.92844) / 3, 3), round((0.89837+0.88655+0.88948) / 3, 3)

(0.93, 0.891)

In [ ]:
# non-tuned yolo 1.2 crop
partial_auroc = (0.93648 + 0.93496 + 0.93751) / 3
sens_at_95_spec =  (0.91618 + 0.89030 + 0.89701) / 3
round(partial_auroc, 3), round(sens_at_95_spec, 3)